# Predicting return from Gold

### Background


Gold has been the original store of value and medium of exchange for mankind for centuries till paper/or fiat currency took over a couple of centuries ago. However, most of the sustainable paper currencyies were backed by Gold as late as 1971, when the Bretton Woods agreement was scrapped and world currencies became a true $'Fiat'$ currency.

Gold however continues to be os interest not only as metal of choice for jewellery, but also as store of value and often advisable part of investment portfolio as it tends to be a hedge and safe haven when economies tend to (or atleat appear to) be in or at brink of collapse.

Currently there are numerous instruments which can give an investor exposure to Gold and they not necessarily need to keep it physically in their vaults. Exchange traded Funds (ETFs) is the most widely used instrument. As of April 2020, a total of **USD175bn** is invested in Gold ETFs across the globe. This was corpus was just **USD24bn in 2008** before the Global Financial Crisis (GFC)

### Approach

In the series we will take different approaches to predict return from Gold prices using Machine learning. We will use supervised learning methods of regression and classification. We will then use Time Series methods. And then finally we will try to integrate them to see of their predictive ppowers increases due to integration.

First we will go the regression route to predict future returns of Gold over next 2 weeks and 3 weeks period. We will do this by using historical returns of different instruments which I beleive impact or likely to impact the outlook towards Gold. The fundamental reason is, I term Gold as a 'reactionary' asset. It has little fundamentals of its own and movement in prices is often is a derivative of how investors view other asset classes (equities and commdities)

## Importing and Preparing Data

For this and subsequent exercises we will need closing price of several instruments for past 10 years . There are various paid (Reuters, Bloomberg) and free resources (IEX, Quandl, Yahoofinance, Google finance) that we can use to either extract and load data in csv or we can directly call their APIs. Since in this project I needed different type of asset classes (Equities, Commodities, Debt and precious metals) I found the 'yahoofinancials' package to be very helpful and straight forward. (https://pypi.org/project/yahoofinancials/)

In [ ]:
!pip3 install pandas
!pip3 install seaborn
!pip3 install datetime
!pip3 install matplotlib
!pip3 install yahoofinancials

In [9]:
#Importing Libraries
import pandas as pd
import seaborn
from datetime import datetime
import matplotlib.pyplot as plt
from yahoofinancials import YahooFinancials

I have prepared a list instruments for which we need to import data. yahoofinancials package requires Yahoo ticker symbols. The list contains the ticker symbols and their descriptions. The excel file containing the list can be found here..... We import that file and extract the tciker symbols and the names as seprarate lists

In [10]:
# ticker_details = pd.read_excel('/Users/vanessawilson/Pbox/Gold-Return-Prediction/Ticker List.xlsx')

ticker_details = pd.read_excel('Ticker List.xlsx')


ticker_details.head(20)

,Ticker,Description
0,GC=F,Gold
1,SI=F,Silver
2,CL=F,Crude Oil
3,^GSPC,S&P500
4,^RUT,Russel 2000 Index
5,ZN=F,10 Yr US T-Note futures
6,ZT=F,2 Yr US T-Note Futures
7,PL=F,Platinum
8,HG=F,Copper
9,DX=F,Dollar Index


In [11]:
ticker = ticker_details['Ticker'].to_list()
names = ticker_details['Description'].to_list()

Once we have the list, we need to define what date range we need to import the data for. The period I have chosen is Jan 2010 till 1st Mar 2020. The reason I did not pull data prior to that is because the GFC in 2008-09 massively changed the economic and market landscapes. Relationships pririo to that peirod might be of less relevance now. We also dont want to feed very less data as the models might tend to overfit.

We create a date-range and write it to an empty dataframe named values where we would extract and past the values we pull from yahoofinancials.

In [ ]:
#Extracting Data from Yahoo Finance and Adding them to Values table using date as key
end_date= "2020-03-01"
start_date = int("2010-01-01")
date_range = pd.bdate_range(start=start_date,end=end_date)
values = pd.DataFrame({ 'Date': date_range})
values['Date']= pd.to_datetime(values['Date'])

Once we have the date range in dataframe, we need to use ticker symbols to pull out data from the API. yahoofinancials returns the output in a JSON format. The following code loops over the the list of ticker symbols and extracts just the closing prices for all the historical dates and keeps them adding to the dataframe horizontally. Note I have used the merge function to mantain the sanctity of dates. Given these asset classes might have different regional and trading holidays, the date ranges are not bound to be the same. By merging, we will eventually have several NAs which we will frontfill later on.

In [ ]:
#Extracting Data from Yahoo Finance and Adding them to Values table using date as key
for i in ticker:
    raw_data = YahooFinancials(i)
    raw_data = raw_data.get_historical_price_data(start_date, end_date, "daily")
    df = pd.DataFrame(raw_data[i]['prices'])[['formatted_date','adjclose']]
    df.columns = ['Date1',i]
    df['Date1']= pd.to_datetime(df['Date1'])
    values = values.merge(df,how='left',left_on='Date',right_on='Date1')
    values = values.drop(labels='Date1',axis=1)

#Renaming columns to represent instrument names rather than their ticker codes for ease of readability
names.insert(0,'Date')
values.columns = names
print(values.shape)
print(values.isna().sum())
values.tail()

In [ ]:
#Front filling the NaN values in the data set
values = values.fillna(method="ffill",axis=0)
values = values.fillna(method="bfill",axis=0)
values.isna().sum()

In [ ]:
# Co-ercing numeric type to all columns except Date
cols=values.columns.drop('Date')
values[cols] = values[cols].apply(pd.to_numeric,errors='coerce').round(decimals=1)
values.tail()

In [ ]:
values.to_csv("Training Data_Values.csv")

In approach we highlighted that we will used lagged returns of the listed instruments to predict future returns on Gold. Here we go on to calculate shortterm returns of all the instruments and longer term returns of few selected instruments. The fundamental idea behind it is, that if a certain asset has highly outperformed or underperformed, there is greater likelihood of portfolio rebalancing which would impact returns on other asset clasees. Eg: If the stock markets (S&P500) has shown stupendous returns in past 6 months, asset managers might want to book profits and allocate some funds to say precious metals and prepare for stock market correction. We will however, use Machine Learning to evaluate the hypothesis.

In [ ]:
imp = ['Gold','Silver', 'Crude Oil', 'S&P500','MSCI EM ETF']
# Calculating Short term -Historical Returns
change_days = [1,3,5,14,21]

data = pd.DataFrame(data=values['Date'])
for i in change_days:
    print(data.shape)
    x= values[cols].pct_change(periods=i).add_suffix("-T-"+str(i))
    data=pd.concat(objs=(data,x),axis=1)
    x=[]
print(data.shape)

# Calculating Long term Historical Returns
change_days = [60,90,180,250]

for i in change_days:
    print(data.shape)
    x= values[imp].pct_change(periods=i).add_suffix("-T-"+str(i))
    data=pd.concat(objs=(data,x),axis=1)
    x=[]
print(data.shape)

Besides just the lagged returns, we also see how far the current Gold price is from its moving average for with different window. This is a very commonly used metric in technical analysis where moving averages offer supports and resistances for asset prices. We use a combination of simple and exponential moving averages. We then add these moving averages to the existing feature space.

In [ ]:
#Calculating Moving averages for Gold
moving_avg = pd.DataFrame(values['Date'],columns=['Date'])
moving_avg['Date']=pd.to_datetime(moving_avg['Date'],format='%Y-%b-%d')
moving_avg['Gold/15SMA'] = (values['Gold']/(values['Gold'].rolling(window=15).mean()))-1
moving_avg['Gold/30SMA'] = (values['Gold']/(values['Gold'].rolling(window=30).mean()))-1
moving_avg['Gold/60SMA'] = (values['Gold']/(values['Gold'].rolling(window=60).mean()))-1
moving_avg['Gold/90SMA'] = (values['Gold']/(values['Gold'].rolling(window=90).mean()))-1
moving_avg['Gold/180SMA'] = (values['Gold']/(values['Gold'].rolling(window=180).mean()))-1
moving_avg['Gold/90EMA'] = (values['Gold']/(values['Gold'].ewm(span=90,adjust=True,ignore_na=True).mean()))-1
moving_avg['Gold/180EMA'] = (values['Gold']/(values['Gold'].ewm(span=180,adjust=True,ignore_na=True).mean()))-1
moving_avg = moving_avg.dropna(axis=0)
print(moving_avg.shape)
moving_avg.head()

In [ ]:
#Merging Moving Average values to the feature space
print(data.shape)
data['Date']=pd.to_datetime(data['Date'],format='%Y-%b-%d')
data = pd.merge(left=data,right=moving_avg,how='left',on='Date')
print(data.shape)
data.isna().sum()

This wall all about features. Now we need to create targets, i.e what we want to predict. Since we are predicting returns, we need to pick a horizon for which we need to predict returns. I have chosen 14-day and 22-day horizons because other smaller horizons tend to be very volatile and lack and predictive power. One can however, experiment with other horizons as well.

In [ ]:
#Caluculating forward returns for Target
y = pd.DataFrame(data=values['Date'])
print(y.shape)
y['Gold-T+14']=values["Gold"].pct_change(periods=-14)
y['Gold-T+22']=values["Gold"].pct_change(periods=-22)
print(y.shape)
y.isna().sum()

In [ ]:
# Removing NAs
print(data.shape)
data = data[data['Gold-T-250'].notna()]
y = y[y['Gold-T+22'].notna()]
print(data.shape)
print(y.shape)

Now we will merge the Target variables with the feature space to get a data whcih we can finally start modelling on.

In [ ]:
#Adding Target Variables
data = pd.merge(left=data,right=y,how='inner',on='Date',suffixes=(False,False))
print(data.shape)
data.isna().sum()

In [ ]:
data.to_csv("Training Data.csv",index=False)

In [ ]:
corr = data.corr().iloc[:,-2:].drop(labels=['Gold-T+14','Gold-T+22'],axis=0)

In [ ]:
import seaborn as sns
import numpy as np

In [ ]:
sns.distplot(corr.iloc[:,0])

In [ ]:
pd.set_option('display.max_rows', None)
corr_data = data.tail(2000).corr()
corr_data = pd.DataFrame(corr_data['Gold-T+14'])
#corr_data = corr_data.iloc[3:,]
corr_data = corr_data.sort_values('Gold-T+14',ascending=False)
#corr_data

In [ ]:
sns.distplot(corr_data)

# Regression

## 22 Day Model

In [ ]:
#data = pd.read_csv("Training Data.csv")

In [ ]:
from pycaret.regression import *

In [ ]:
data_22= data.drop(['Gold-T+14'],axis=1)
data_22.head()

In [ ]:
a=setup(data_22,target='Gold-T+22',
        ignore_features=['Date'],session_id=11,
        silent=True,profile=False,remove_outliers=False);
#transformation=True,
#pca=True,pca_method='kernel',
#pca_components=10,
#create_clusters=True,
#cluster_iter=10,
#feature_ratio=True,
#normalize=True,
#transform_target=True,
#silent=True);

In [ ]:
compare_models(blacklist=['tr','ard'],turbo=True)

In [ ]:
knn_tuned = tune_model('knn',n_iter=150)

In [ ]:
catb_tuned = tune_model('catboost')

In [ ]:
et_tuned = tune_model('et')

In [ ]:
et = create_model('et')

In [ ]:
evaluate_model(knn_tuned)

In [ ]:
evaluate_model(et)

In [ ]:
b=setup(data_22,target='Gold-T+22',
        ignore_features=['Date'],session_id=11,
        silent=True,profile=False,remove_outliers=True);
#transformation=True,
#pca=True,pca_method='kernel',
#pca_components=10,
#create_clusters=True,
#cluster_iter=10,
#feature_ratio=True,
#normalize=True,
#transform_target=True,
#silent=True);

In [ ]:
knn_tuned = tune_model('knn',n_iter=150)

In [ ]:
et = create_model('et')

In [ ]:
catb = create_model('catboost')

### Ensembling Models

In [ ]:
et_bagged = ensemble_model(et,method='Bagging')

In [ ]:
knn_tuned_bagged = ensemble_model(knn_tuned, method='Bagging')

### Blending Models

In [ ]:
blend_knn_et = blend_models(estimator_list=[knn_tuned,et])

In [ ]:
stack1 = create_stacknet(estimator_list=[[catb,knn_tuned],[et,blend_knn_et]],restack=True)

In [ ]:
stack2 = create_stacknet(estimator_list=[[catb,et,knn_tuned],[blend_knn_et]], restack=True)

In [ ]:
stack3 = create_stacknet(estimator_list=[[catb,et,knn_tuned],[blend_knn_et]], restack=True,meta_model=blend_knn_et)

In [ ]:
save_model(model=stack2, model_name='22Day Regressor')

### 14 Day Model 

In [ ]:
data_14= data.drop(['Gold-T+22'],axis=1)
data_14.head()

In [ ]:
c=setup(data_14,target='Gold-T+14',
        ignore_features=['Date'],session_id=11,
        silent=True,profile=False,remove_outliers=True);
#transformation=True,
#pca=True,pca_method='kernel',
#pca_components=10,
#create_clusters=True,
#cluster_iter=10,
#feature_ratio=True,
#normalize=True,
#transform_target=True,
#silent=True);

In [ ]:
compare_models(blacklist=['tr','ard'],turbo=True)

In [ ]:
knn_tuned = tune_model('knn',n_iter=150)
catb = create_model('catboost')
et = create_model('et')
knn_tuned_bagged = ensemble_model(knn_tuned, method='Bagging')
blend_knn_et = blend_models(estimator_list=[knn_tuned,et])

In [ ]:
stack1 = create_stacknet(estimator_list=[[catb,knn_tuned],[et,blend_knn_et]],restack=True)

In [ ]:
stack2 = create_stacknet(estimator_list=[[catb,et,knn_tuned],[blend_knn_et]], restack=True)

In [ ]:
stack3 = create_stacknet(estimator_list=[[catb,et,knn_tuned],[blend_knn_et]], restack=True,meta_model=blend_knn_et)

In [ ]:
save_model(model=stack2, model_name='14Day Regressor')